In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
import subprocess
from copy import deepcopy
from time import time
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdDistGeom import EmbedMultipleConfs
from linux_qm.src.util import _load_smiles3D, _create_tmp_dir, draw3Dconfs, SetPositions
from linux_qm.qm.crest.crest import conformer_pipeline


ORCA_TMP = '.orca_tmp'

In [3]:
smi = 'COC1CN(C)C1'
# smi = 'CO'


# mol = _load_smiles3D(smi)
# EmbedMultipleConfs(mol, numConfs=3)

mol = conformer_pipeline(smi)

# Get the conformers

conformers = mol.GetConformers()

# Print the number of conformers generated
print(f"Number of conformers generated: {len(conformers)}")
print("Energies:", [c.GetProp('energy') for c in conformers])

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_INVALID_FLAG IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


Number of conformers generated: 4
Energies: ['-23.275472730000001', '-23.273513529999999', '-23.273084449999999', '-23.272812779999999']


In [4]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [5]:
### QM calculation by ORCA for each conformer

def gen_input(conf, options: dict):
    method, solvent = options['method'], options.get('solvent')
    geom_maxiter, n_jobs = options.get('geom_maxiter', 100), options.get('n_jobs', 8)

    input_str =""
    input_str += f"!{method}\n"
    input_str += f"%geom MaxIter {geom_maxiter} end\n"
    if solvent:
        input_str += (f"%cpcm\n"
        f"  smd true\n"
        f"  SMDsolvent \"{solvent}\"\n"
        f"  end\n")
    
    # input_str += (f"%output\n "
    #     f"  PrintLevel Huge\n"
    #     # f"  Print[P_MOs] 1\n"
    #     # f"  Print[ P_gradient ] 1\n"
    #     f"  end\n")    
    input_str += f"%pal nprocs {n_jobs} end\n"
    input_str += gen_xyz_block(conf)
    return input_str

def gen_xyz_block(conf):
    mol = conf.GetOwningMol()
    xyz_block = "*xyz 0 1\n"
    for i in range(conf.GetNumAtoms()):
        symbol = mol.GetAtomWithIdx(i).GetSymbol()
        x, y, z = conf.GetAtomPosition(i)
        xyz_block += f"{symbol:3}{x:20.8f}{y:20.8f}{z:20.8f}\n"
    xyz_block += "*\n"
    return xyz_block

def write_input(content):
    fname = 'task.inp'
    with open(fname, 'w') as f:
        f.write(content)
    return fname

def write_output(content):
    fname = 'output'
    with open(fname, 'w') as f:
        f.write(content)
    return fname

options = {
    'method': 'B3LYP DEF2-SVP OPT',
    'solvent': 'THF',
    'nmr_atoms': [0],
    'geom_maxiter': 100,
    'n_jobs': 22,
}    

conf = mol.GetConformer()
print(gen_input(conf, options))

!B3LYP DEF2-SVP OPT
%geom MaxIter 100 end
%cpcm
  smd true
  SMDsolvent "THF"
  end
%pal nprocs 22 end
*xyz 0 1
C           -2.71478820         -0.42885540          0.15406937
O           -1.61467692          0.39694184          0.44575716
C           -0.56782235          0.27296990         -0.47466576
C            0.36187249         -0.95002327         -0.33959139
N            1.51499860         -0.04688833         -0.36976593
C            2.58121940         -0.29507022          0.56449033
C            0.66125759          1.11992193         -0.13116444
H           -3.12947443         -0.19821137         -0.83506216
H           -3.46298983         -0.22811082          0.91812373
H           -2.43967404         -1.48951654          0.18103452
H           -0.93022880          0.41958554         -1.50069853
H            0.36201809         -1.68034166         -1.15006164
H            0.22254057         -1.45665655          0.62351908
H            3.31303418          0.50745220          0.4

In [6]:
def set_orca_env():
    ORCA_PATH = "/opt/orca-5.0.3"
    os.environ['PATH'] = f"{ORCA_PATH}:{os.environ.get('PATH')}"
    os.environ['LD_LIBRARY_PATH'] = f"{ORCA_PATH}:{os.environ.get('LD_LIBRARY_PATH', '')}"
    # supposed to be path
    os.environ['PATH'] = f"/opt/openmpi-4.1.1/bin:{os.environ['PATH']}"
    os.environ['LD_LIBRARY_PATH'] = f"/opt/openmpi-4.1.1/lib:{os.environ['LD_LIBRARY_PATH']}"
    
    # intel MKL
    os.environ['LD_LIBRARY_PATH'] = f"/usr/lib/x86_64-linux-gnu:{os.environ['LD_LIBRARY_PATH']}"
    
    # nbo
    NBOBIN = '/opt/NBO6'
    os.environ['NBOEXE'] = f"{NBOBIN}/nbo6.exe"
    os.environ['GENEXE'] = f"{NBOBIN}/gennbo6.exe"
    os.environ['PATH'] = f"{NBOBIN}:{os.environ['PATH']}"

    # real openmpi path
    # os.environ['LD_LIBRARY_PATH'] = "/opt/orca-5.0.3/openmpi-4.1.1/lib:/opt/orca-5.0.3/orca:"
    # os.environ['PATH'] = f"/opt/orca-5.0.3/openmpi-4.1.1/bin:{os.environ['PATH']}"

if 'orca' not in os.environ['PATH']:
    set_orca_env()

In [7]:
os.environ['LD_LIBRARY_PATH']

'/usr/lib/x86_64-linux-gnu:/opt/openmpi-4.1.1/lib:/opt/orca-5.0.3:'

In [8]:
import cclib

In [9]:
saved_work_dir = os.getcwd()
tmp_path = _create_tmp_dir(ORCA_TMP)
os.chdir(tmp_path)

data = None

options = {
    'method': 'B3LYP  6-31G',
    # 'method': 'B3LYP  6-31++G(d,p) OPT',
    'solvent': None,
    'geom_maxiter': 100,
    'n_jobs': 16,
}

input_str = gen_input(conf, options)
fname = write_input(input_str)
    
res = subprocess.run(
    ['/opt/orca-5.0.3/orca', fname, '--use-hwthread-cpus'], 
    capture_output=True,
    text=True
)

fname = write_output(res.stdout)

# parse ORCA output
data = cclib.io.ccread(fname)    

# resotore
os.chdir(saved_work_dir)
shutil.rmtree(tmp_path)

if not data.metadata['success']:
    print('Failed')

[23-11-2023 07:18:02] [INFO] Identified logfile to be in ORCA format


In [10]:
def update_conformer(conf, cclib_data):
    SetPositions(conf, cclib_data.atomcoords[-1])
    conf.SetDoubleProp('energy', cclib_data.scfenergies[-1])


def run_orca(conf, options):
    # save current dir
    saved_work_dir = os.getcwd()
    tmp_path = _create_tmp_dir(ORCA_TMP)
    os.chdir(tmp_path)    
        
    input_str = gen_input(conf, options)
    fname = write_input(input_str)
        
    output = subprocess.run(
        ['/opt/orca-5.0.3/orca', fname, '--use-hwthread-cpus'], 
        capture_output=True,
        text=True
    ).stdout
    
    fname = write_output(output)
    
    # parse ORCA output
    data = cclib.io.ccread(fname)    
    
    # restore
    os.chdir(saved_work_dir)
    shutil.rmtree(tmp_path)
    
    success = data.metadata['success']
    conf.SetBoolProp('success', success)

    # DEV
    print('Success:', success)
    
    # update conformer
    if success:
        update_conformer(conf, data)
        # DEV
        print('Elapsed time:', data.metadata['cpu_time'][0])
        print('Num iter:', len(data.atomcoords))
    else:
        print(output)
    
    return data, output




# options = {
#     'method': 'M062X 6-311++G(d,p) OPT NMR',
#     'solvent': 'THF',
#     'geom_maxiter': 100,
#     'n_jobs': 16,
# }


# for conf in deepcopy(mol).GetConformers():
#     print(f'======= Conformer {conf.GetId()} ======= ')
#     data, output = run_orca(conf, options)
#     break
# single_point(conf, options)

In [11]:
# query = Chem.MolFromSmarts('C1CCN1')
# 
# atom_ids = mol.GetSubstructMatch(query)
# print(atom_ids)
# AllChem.AlignMolConformers(mol, atomIds=atom_ids)
# 
# draw3Dconfs(mol, autoalign=False)

In [12]:
%%time
options = {    
    'solvent': 'THF',
    'geom_maxiter': 100,
    'n_jobs': 16,
}

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X 6-311++G(d,p) OPT NMR'
data, output = run_orca(conf, options)

[23-11-2023 07:20:20] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:02:17.911000
Num iter: 9
CPU times: user 140 ms, sys: 46.8 ms, total: 187 ms
Wall time: 2min 17s


In [13]:
%%time

conf = deepcopy(mol).GetConformer()

options['method'] = 'M062X 6-31G(d,p) OPT'
run_orca(conf, options)

options['method'] = 'M062X 6-311++G(d,p) OPT NMR'
data, output = run_orca(conf, options)

[23-11-2023 07:21:36] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:01:16.214000
Num iter: 9


[23-11-2023 07:22:54] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:01:17.614000
Num iter: 5
CPU times: user 183 ms, sys: 96 ms, total: 279 ms
Wall time: 2min 33s


In [44]:
%%time

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X def2-TZVP OPT'
data, output = run_orca(conf, options)

[23-11-2023 08:08:50] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:02:26.726000
Num iter: 9
CPU times: user 141 ms, sys: 31 ms, total: 172 ms
Wall time: 2min 26s


In [37]:
%%time

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X aug-cc-pVDZ OPT'
data, output = run_orca(conf, options)

[23-11-2023 07:40:13] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:02:58.253000
Num iter: 9
CPU times: user 143 ms, sys: 32.7 ms, total: 176 ms
Wall time: 2min 58s


In [41]:
%%time

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X cc-pVTZ OPT'
data, output = run_orca(conf, options)

[23-11-2023 07:54:31] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:03:20.218000
Num iter: 9
CPU times: user 129 ms, sys: 66.7 ms, total: 196 ms
Wall time: 3min 20s


In [42]:
%%time

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X aug-cc-pVTZ OPT'
data, output = run_orca(conf, options)

[23-11-2023 08:03:34] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:09:02.365000
Num iter: 9
CPU times: user 160 ms, sys: 83.4 ms, total: 243 ms
Wall time: 9min 2s


In [45]:
%%time

conf = deepcopy(mol).GetConformer()
options['method'] = 'M062X cc-pVDZ OPT'
data, output = run_orca(conf, options)

options['method'] = 'M062X aug-cc-pVTZ OPT'
data, output = run_orca(conf, options)

[23-11-2023 08:30:21] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:01:21.496000
Num iter: 9


[23-11-2023 08:35:16] [INFO] Identified logfile to be in ORCA format


Success: True
Elapsed time: 0:04:55.687000
Num iter: 5
CPU times: user 215 ms, sys: 92.4 ms, total: 308 ms
Wall time: 6min 17s


In [43]:
query = Chem.MolFromSmarts('C1CCN1')
atom_ids = mol.GetSubstructMatch(query)
print(atom_ids)
AllChem.AlignMolConformers(mol, atomIds=atom_ids)
draw3Dconfs(mol, confIds=[0], autoalign=False)

(3, 2, 6, 4)
num conformers 4


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [52]:
data.scfvalues

[array([[ 0.00000000e+00,  1.75780800e-01,  1.13529000e-03],
        [-7.83250489e-02,  1.44294260e-01,  9.00510000e-04],
        [-3.82209220e-02,  3.60672850e-01,  2.12364000e-03],
        [-5.95070839e-02,  3.45931600e-02,  3.53380000e-04],
        [-1.10323800e-02,  1.38156100e-02,  1.32650000e-04],
        [-2.43487796e-04,  4.96315000e-03,  5.04600000e-05],
        [-3.80466000e-05,  2.50000000e-04,  7.07300000e-03],
        [-5.39920000e-06,  1.49000000e-04,  3.84100000e-03],
        [-3.79500000e-07,  6.80000000e-05,  8.82000000e-04],
        [-4.23500000e-07,  1.50000000e-05,  6.69000000e-04],
        [-8.50000000e-09,  7.00000000e-06,  7.70000000e-05],
        [-5.17330000e-09,  3.99680000e-15,  1.17560000e-17]]),
 array([[-3.27054152e+02,  2.26400000e-03,  6.88800000e-03],
        [-3.97856500e-04,  4.46000000e-04,  5.24900000e-03],
        [-3.05938000e-05,  2.93000000e-04,  2.26000000e-03],
        [-1.46754000e-05,  6.50000000e-05,  7.32000000e-04],
        [-2.08900000e-

In [51]:
data.scftargets

array([[1.e-08, 1.e-07, 5.e-09],
       [1.e-08, 1.e-07, 5.e-09],
       [1.e-08, 1.e-07, 5.e-09],
       [1.e-08, 1.e-07, 5.e-09],
       [1.e-08, 1.e-07, 5.e-09]])

In [36]:
# NMR shielding
for k,v in data.nmrtensors.items():
    print(mol.GetAtomWithIdx(k).GetSymbol(), k, v['isotropic'])

C 0 125.508
O 1 275.301
C 2 110.446
C 3 114.12
N 4 223.565
C 5 133.679
C 6 115.657
H 7 28.766
H 8 28.318
H 9 28.777
H 10 27.965
H 11 28.106
H 12 29.335
H 13 29.221
H 14 30.004
H 15 29.269
H 16 29.104
H 17 28.096


In [15]:
dir(data)

['OPT_DONE',
 'OPT_NEW',
 'OPT_UNCONVERGED',
 'OPT_UNKNOWN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_attrlist',
 '_dictsofarrays',
 '_intarrays',
 '_listsofarrays',
 'arrayify',
 'atomcharges',
 'atomcoords',
 'atommasses',
 'atomnos',
 'charge',
 'check_values',
 'closed_shell',
 'converged_geometries',
 'coreelectrons',
 'geotargets',
 'geovalues',
 'getattributes',
 'grads',
 'homos',
 'listify',
 'metadata',
 'moenergies',
 'moments',
 'mosyms',
 'mult',
 'natom',
 'nbasis',
 'nelectrons',
 'new_geometries',
 'nmo',
 'nmrtensors',
 'optdone',
 'scfenergies',
 'scftargets',
 'scfvalues',
 'setattributes',
 'typecheck',
 'unconverged_geometr

In [16]:
print(data.metadata['cpu_time'][0])

0:01:17.614000


In [17]:
len(data.converged_geometries)

5

In [18]:
dir(data)

['OPT_DONE',
 'OPT_NEW',
 'OPT_UNCONVERGED',
 'OPT_UNKNOWN',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_attributes',
 '_attrlist',
 '_dictsofarrays',
 '_intarrays',
 '_listsofarrays',
 'arrayify',
 'atomcharges',
 'atomcoords',
 'atommasses',
 'atomnos',
 'charge',
 'check_values',
 'closed_shell',
 'converged_geometries',
 'coreelectrons',
 'geotargets',
 'geovalues',
 'getattributes',
 'grads',
 'homos',
 'listify',
 'metadata',
 'moenergies',
 'moments',
 'mosyms',
 'mult',
 'natom',
 'nbasis',
 'nelectrons',
 'new_geometries',
 'nmo',
 'nmrtensors',
 'optdone',
 'scfenergies',
 'scftargets',
 'scfvalues',
 'setattributes',
 'typecheck',
 'unconverged_geometr

In [19]:
data.optdone

True

In [20]:
data.metadata

{'package': 'ORCA',
 'methods': ['DFT', 'DFT', 'DFT', 'DFT', 'DFT'],
 'success': True,
 'legacy_package_version': '5.0.3',
 'package_version': '5.0.3',
 'basis_set': '6-311++G(d,p)',
 'warnings': ['Geometry OptimizationSwitching off AutoStartFor restart on a previous wavefunction, please use MOREAD',
  'Minnesota functionals are quite sensitive to the integration grid.see SE Wheeler, KN Houk, JCTC 2010, 6, 395,N Mardirossian, M Head-Gordon, JCTC 2016, 12, 4303.DEFGRID3 seems to be a minimum grid for reliable results with these functionals!Please increase the integration grid!'],
 'info': ['the flag for use of the SHARK integral package has been found!'],
 'input_file_name': 'task.inp',
 'input_file_contents': '!M062X 6-311++G(d,p) OPT NMR\n%geom MaxIter 100 end\n%cpcm\n  smd true\n  SMDsolvent "THF"\n  end\n%pal nprocs 16 end\n*xyz 0 1\nC           -2.73166000         -0.42891300          0.06568100\nO           -1.69258800          0.47020300          0.38413500\nC           -0.584961

In [21]:
data.new_geometries

array([[[-2.73166 , -0.428913,  0.065681],
        [-1.692588,  0.470203,  0.384135],
        [-0.584961,  0.294699, -0.444423],
        [ 0.318446, -0.937096, -0.217537],
        [ 1.481163, -0.047712, -0.386694],
        [ 2.63364 , -0.299075,  0.451037],
        [ 0.645131,  1.118235, -0.052189],
        [-3.11054 , -0.260819, -0.952362],
        [-3.541348, -0.262969,  0.778925],
        [-2.400831, -1.474237,  0.143859],
        [-0.856075,  0.412823, -1.502824],
        [ 0.270658, -1.784797, -0.907525],
        [ 0.201829, -1.289001,  0.821651],
        [ 3.376249,  0.492224,  0.308155],
        [ 2.374666, -0.342195,  1.523777],
        [ 3.093524, -1.251502,  0.169314],
        [ 0.615582,  1.32106 ,  1.030892],
        [ 0.87263 ,  2.037938, -0.598603]],

       [[-2.732772, -0.433447,  0.065903],
        [-1.697911,  0.474342,  0.37912 ],
        [-0.585376,  0.300297, -0.445689],
        [ 0.315233, -0.934156, -0.217946],
        [ 1.481057, -0.046769, -0.385799],
        [

In [22]:
# HOMO LUMO
homo = data.moenergies[0][data.homos[0]]
lumo = data.moenergies[0][data.homos[0] + 1]
homo, lumo

(-7.7628095042939, 0.14092776317395)

In [23]:
print(data.metadata['cpu_time'][0])

0:01:17.614000


In [24]:
data.natom, data.nmo

(18, 231)

In [25]:
data.metadata

{'package': 'ORCA',
 'methods': ['DFT', 'DFT', 'DFT', 'DFT', 'DFT'],
 'success': True,
 'legacy_package_version': '5.0.3',
 'package_version': '5.0.3',
 'basis_set': '6-311++G(d,p)',
 'warnings': ['Geometry OptimizationSwitching off AutoStartFor restart on a previous wavefunction, please use MOREAD',
  'Minnesota functionals are quite sensitive to the integration grid.see SE Wheeler, KN Houk, JCTC 2010, 6, 395,N Mardirossian, M Head-Gordon, JCTC 2016, 12, 4303.DEFGRID3 seems to be a minimum grid for reliable results with these functionals!Please increase the integration grid!'],
 'info': ['the flag for use of the SHARK integral package has been found!'],
 'input_file_name': 'task.inp',
 'input_file_contents': '!M062X 6-311++G(d,p) OPT NMR\n%geom MaxIter 100 end\n%cpcm\n  smd true\n  SMDsolvent "THF"\n  end\n%pal nprocs 16 end\n*xyz 0 1\nC           -2.73166000         -0.42891300          0.06568100\nO           -1.69258800          0.47020300          0.38413500\nC           -0.584961

In [26]:
data.scfenergies

array([-8898.73503472, -8898.73716429, -8898.73758198, -8898.73766933,
       -8898.73770797])